# Example Genetic Algorithm

Taken from https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/

## Binary optimization

In [ ]:
# genetic algorithm search of the one max optimization problem
import numpy as np
from numpy.random import randint
from numpy.random import rand

# objective function
def onemax(x):
    score_ = np.array([-1,3,10,4,-1,-5,-1,-1,-1,-1,-1,-1,-7,3,-1,-1,-1,-1,-1,-1])
    # run powerflow
    # check drop
    if sum(x[1:4]) == 3:
        return 1
    else:
        return np.dot(x,score_)

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)] # random 0 or 1, length = n_bits, total n_pop set
    # keep track of best solution
    best, best_eval = 0, objective(pop[0]) # initial best, could be skipped and default value can be entered
    # enumerate generations
    for gen in range(n_iter): # main loop
        # evaluate all candidates in the population
        scores = [objective(c) for c in pop] # evaluation to be used for selection
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)] # in some approach, you can make previous best solution always survive to next gen
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# define the total iterations
n_iter = 25
# bits
n_bits = 20
# define the population size
n_pop = 50
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)
# perform the genetic algorithm search
best, score = genetic_algorithm(onemax, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>0, new best f([1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0]) = -14.000
>1, new best f([1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1]) = -16.000
>1, new best f([1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]) = -17.000
>2, new best f([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1]) = -18.000
>3, new best f([1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1]) = -19.000
>4, new best f([1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1]) = -21.000
>4, new best f([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1]) = -22.000
>5, new best f([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]) = -23.000
>8, new best f([1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1]) = -24.000
>9, new best f([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1]) = -25.000
>12, new best f([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]) = -26.000
Done!
f([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 

## Continuous Function Optimization

In [ ]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

# objective function
def objective(x):
    return x[0]**2.0 + x[1]**2.0

# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
    for gen in range(n_iter):
        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
                print(f"{gen}, new best {pop[i]}")
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval, pop]

# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# define the total iterations
n_iter = 25
# bits per variable
n_bits = 16
# define the population size
n_pop = 50
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score, pop = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([-1.472320556640625, 0.62469482421875]) = 2.557971
0, new best [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
>0, new best f([-0.549468994140625, -1.206817626953125]) = 1.758325
0, new best [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1]
>0, new best f([1.104583740234375, 0.22979736328125]) = 1.272912
0, new best [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0]
>2, new best f([-0.282745361328125, -0.961761474609375]) = 1.004930
2, new best [0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1]
>2, new best f([-0.48309326171875, -0.619354248046875]) = 0.616979
2, new best [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]
>3, new best f([-0.55145263671875, -0.137481689453125]) = 0.323001
3, new best [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,

## Matpower via `mypower` (distributed generator placement)

Install mypower from `https://github.com/yasirroni/mypower`. Don't forget to leave a star.

In [ ]:
import mypower as myp
from mypower.oc_api import oc_matpower
import numpy as np

In [ ]:
# initial
oc = oc_matpower()
case_name = 'case9'
mypc = oc.eval(case_name, verbose=False)
idx = myp.get_index()

candidate_generator_capacity = np.array([10, 20, 40, 80])

In [ ]:
# losses before distributed generator placements
mypc = oc.runpf(mypc, verbose=False)
losses = sum(np.abs(mypc.branch[:,idx['PF']] + mypc.branch[:,idx['PT']]))
losses

4.6410214744828835

In [ ]:
# genetic algorithm search of the one max optimization problem
import numpy as np
from numpy.random import randint
from numpy.random import rand
import copy

# objective function
def generator_placement(x):
    # binary to location
    x_ = np.array(decode(x, n_bits_each)) + 1 # shift 1 bus to start from bus 2

    # run powerflow
    mypc_ = copy.deepcopy(mypc)
    for i, x__ in enumerate(x_):
        mypc_['bus'][x__,idx['PD']] = mypc_['bus'][x__,idx['PD']] - candidate_generator_capacity[i]

    # check losses
    mypc_ = oc.runpf(mypc_, verbose=False)
    losses = sum(np.abs(mypc_.branch[:,idx['PF']] + mypc_.branch[:,idx['PT']]))

    return losses

def decode(bitstring, n_bits_each):
    decoded = []
    n_group = len(bitstring) // n_bits_each
    for i in range(n_group):
        # extract the substring
        start, end = i * n_bits_each, (i * n_bits_each) + n_bits_each
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # store
        decoded.append(integer)
    return decoded

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)] # random 0 or 1, length = n_bits, total n_pop set
    # keep track of best solution
    best, best_eval = 0, objective(pop[0]) # initial best, could be skipped and default value can be entered
    # enumerate generations
    for gen in range(n_iter): # main loop
        # evaluate all candidates in the population
        scores = [objective(c) for c in pop] # evaluation to be used for selection
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)] # in some approach, you can make previous best solution always survive to next gen
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# define the total iterations
n_iter = 25
# bits
n_bits_each = 3
n_bits_group = 4
n_bits = n_bits_each * n_bits_group # 3 bits = 7 candidate bus, 4 bits group = 4 generator
# define the population size
n_pop = 50
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)
# perform the genetic algorithm search
best, score = genetic_algorithm(generator_placement, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))
best_location = np.array(decode(best, n_bits_each))+1
print(f'Best location to install generator {candidate_generator_capacity} is in {best_location}')

>0, new best f([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]) = 5.527
>0, new best f([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0]) = 4.802
>2, new best f([1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0]) = 4.525
>2, new best f([0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0]) = 4.453
>3, new best f([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]) = 4.388
>4, new best f([0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0]) = 4.328
>8, new best f([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0]) = 4.316
Done!
f([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0]) = 4.315767
Best location to install generator [10 20 40 80] is in [4 4 8 3]


## What you can do to customize genetic algorithm?

1. Make some initial population deterministic on interesting points (e.g. highest possible value, lowest possible value)
2. Warm start initial population using other fast deterministic method (e.g. naive method)
3. Change crossover and mutation method into problem specific approach (make your problem become heuristic instead of meta-heuristic)
4. Try to always save best solution, skipping selection method
5. Add penalty, either constant, discrete, or continues function
6. Make generation number very high, but use early termination
7. Use meta approach to chose probability value for mutation, crossover, etc.